# 1. SETTINGS

In [ ]:
# import packages
import pandas as pd
import numpy as np
import scipy.stats
import os

In [ ]:
# pandas options
pd.set_option('display.max_columns', None)

In [ ]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# garbage collection
import gc
gc.enable()

In [ ]:
# helper functions
import functions
from functions import *

# 2. IMPORT

In [ ]:
# import data
train = pd.read_csv('../data/X_train.csv', sep = ',', decimal = '.', error_bad_lines = False)
y     = pd.read_csv('../data/y_train.csv', sep = ',', decimal = '.', error_bad_lines = False)
test  = pd.read_csv('../data/X_test.csv',  sep = ',', decimal = '.', error_bad_lines = False)
print(train.shape)
print(y.shape)
print(test.shape)

In [ ]:
# check data
train.head()

In [ ]:
# check data
test.head()

# 3. MERGER

In [ ]:
# target variable
train['duration'] = y['duration']
test['duration']  = np.nan

In [ ]:
# align columns
train = train.reindex_axis(sorted(train.columns), axis = 1)
test  = test.reindex_axis(sorted(test.columns),   axis = 1)

# check equalty
train.columns == test.columns

In [ ]:
# concatenate
df = pd.concat([train, test], axis = 0)
df = df.reset_index(drop = True)
del train, test
print(df.shape)

In [ ]:
# check distributions
df.describe()

# 4. PROCESSING

### DROP IRRELEVANT FEATURES

In [ ]:
# remove columns with a single value
print(df.shape)
df = df.loc[:, df.nunique(dropna = False) != 1]
print(df.shape)

In [ ]:
# remove irrelevant columns
print(df.shape)
drops = ['image_url']
for var in drops:
    del df[var]
print(df.shape)

### MISSING VALUES

In [ ]:
# check missings
count_missings(df)

In [ ]:
# impute NAs with 0
nas = []
for var in nas:
    df[var].fillna(0, inplace = True)

In [ ]:
# impute missings with means
#target = df['duration'].copy()
#df = df.replace([np.inf, -np.inf], np.nan)
#means = df.mean(axis = 0)
#df.fillna(means, inplace = True)
#df['duration'] = target

### VARIABLE TYPES

In [ ]:
# check data types
df.dtypes

In [ ]:
# convert float to integers
to_int = []
for col in to_int:
    df[col] = df[col].astype('int64')
    
# convert boolean to objects
bools = []
for var in bools:
    df[var] = df[var].astype('object')
    
# convert string to dates
dates = []
for var in dates:
    df[var] = pd.to_datetime(df[var].astype('str'), infer_datetime_format = True)

### AGGREGATIONS

In [ ]:
# aggregated data
stats = ['mean', 'std', 'min', 'max']
df_agg = aggregate_data(df, group_var = 'category', num_stats = stats, var_label = 'category')
df_agg.head()

In [ ]:
# remove target aggregations
drops = ['duration', 'id']
for var in drops:
    vars_to_drop = [var + '_' + s for s in stats]
    for var_to_drop in vars_to_drop:
        del df_agg[var_to_drop]
df_agg.shape

In [ ]:
# merge to the data
print(df.shape)
df = df.merge(df_agg, how = 'left', on = 'category')
print(df.shape)

# 5. WORKING WITH FEATURES

### EXTRACT FEATURES FROM TEXT

In [ ]:
# compute features
text_vars = ['product_description', 'product_name', 'store_name', 'category', 'material',
             'sub_category_1', 'sub_category_2', 'sub_category_3', 'sub_category_4']
print(df.shape)
df = add_text_features(df, strings = text_vars, k = 5, keep = True)
print(df.shape)

### CREATE NEW FEATURES

In [ ]:
df['image_pixels'] = df['image_height'] * df['image_width']

In [ ]:
df['product_volume'] = df['product_height'] * df['product_length'] * df['product_width']

### ENCODE FACTORS

In [ ]:
# convert categorical features
df = encode_factors(df, method = 'label')
df.shape

# 6. EXPORT

In [ ]:
# export CSV
df.to_csv('../data/data_v2.csv', index = False)
df.shape